<a href="https://colab.research.google.com/github/annikaaross/Homochirality-project/blob/annika-branch/Homochirality_with_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homochirality Model

Hello and welcome to Do Opposites Attract?, the first monomer dating show. We're here to see if similarity between member(s) of a relationship matters in the strength of the bond between the individuals. Read on as we introduce you to our monomers, our polymers, and the stylish bag that they reside in. Along the way we will keep you up to date on all the drama of falling in love and breaking hearts.

## Global Parameters

Here's where we keep the rules by which our contestants must play.

In [23]:
#the bond break probbility applied to all bonds
#BASE_BOND_BREAK_PROBABILITY = 0.9
#if a bond itself is homochiral the base bond fator is multiplied by
#this factor
#HOMOCHIRAL_BREAK_FACTOR = 0.9


#this factor is incorporated for each additional neighboring homochiral bond
# becomes less and less influential as the bond gets further away
#HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR = 0.3

#decreases the break probability based on the length of the polymer
#LENGTH_FACTOR = 0.6





#limits the influence of length on breakprobabiltiy(the larger the less influence it has)
#N=40

#saves the break probabilities of sequences of polymers

#used in Poisson distribution
#LAMBDA=1

#################################################################
#these are the same factors as before but with on for left and  #
#one for right homochiral bonds.                                #
#used in biaseastbondbreakability method which can be choosen to#
#be used over the regular eastbondbreakability method in the    #
#self.reset_break_probability() method                          #
#################################################################
#HOMOCHIRAL_BREAK_FACTOR_LEFT = 0.0
#HOMOCHIRAL_BREAK_FACTOR_RIGHT = 0.9
#HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT = 0.3
#HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT = 0.3


N_RANDS = 1000


rands = []


#These are the liklihoods that a monomer will choose to poof or bond, respectively.
#Used in the Reactables class.

#POOF_CHANCE = 0.3333

#BOND_PROB = 0.3333


#POOL_SIZE= 10

#ITERATIONS = 100

# Shorthands #
L = True
R = False

## Imports and Helpers

First we install a package called ```jdc``` which allows us to define classes across different cells using the syntax

```
%%add_to our_class
def our_function(self, our_variable):
  print our_variable
```

The documentation on this magic function can be found at https://alexhagen.github.io/jdc/






In [24]:
!pip install jdc

Here are the imports.

In [25]:
import random
import numpy as np
import copy
from google.colab import widgets, output
import matplotlib.pyplot as plt
import jdc
import more_itertools
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import time
from IPython.display import display, Markdown, clear_output
import ipywidgets

And here are some helper functions.

In [26]:
def make_pool(n):
  """
  Return a list of n new monomers.
  """
  monomers = []
  for i in range(n):
    monomers.append(Monomer())
  return monomers


def numpy_fillna(data):
  """ Rectangularize a jagged array.

  Source: https://stackoverflow.com/a/32043366
  """
  # Get lengths of each row of data
  lens = np.array([len(i) for i in data])

  # Mask of valid places in each row
  mask = np.arange(lens.max()) < lens[:,None]

  # Setup output array and put elements from data into masked positions
  out = np.zeros(mask.shape, dtype=float)
  out[mask] = np.concatenate(data)
  return out

def eAnd(*args): # From https://stackoverflow.com/q/2770434
  return [all(tuple) for tuple in zip(*args)]

def get_rand():
  """A function to provide uniform random numbers"""
  if len(rands)<= 10:
    randsarray = np.random.rand(N_RANDS)
    rands.extend(randsarray.tolist())
  return rands.pop()

def poissonequation(k):
  #checks if value has already been calulated
  if k not in poisson_dict:
    #otherwise calculates the values
    p = np.exp(-LAMBDA)*((LAMBDA**k)/(np.math.factorial(k)))
    #adds to dictionary
    poisson_dict[k] = 1- POISSON_FACTOR*(p)
  return poisson_dict[k]


## The Monomer class

These are our Monomers. The individuals in our dating show. They are the best thing a girl can be in this worl, beautiful little fools. They know that they exist and they know what kind of monomer they are, left or right(thats handedness not political party alignment) What more can you ask for?

### Constructor

In [27]:
class Monomer:

  def __init__(self, **kwargs):
    """
    The constructor method for Monomer objects. Assigns default handedness.
    Accepts argument hand = bool with keyword.
    Considering other syntax for passing in handedness.
    Keeps track of how old the monomer is
    """
    import random
    self._handedness = kwargs['hand'] if 'hand' in kwargs else random.choice([True,False])
    self._eastbrkprob = kwargs['brkprob'] if 'brkprob' in kwargs else -1
    #keeps track of its age, the number of iterations it has survived
    self._age = 0
  
  def __repr__(self):
    return str(f"{self.get_handedness()}-monomer")


### Getters and Setters

In [28]:
%%add_to Monomer

def get_handedness(self):
  """Getter method for Monomer handedness."""
  return self._handedness

def get_age(self):
  """Getter method for Monomer age"""
  return self._age


def get_eastbrkprob(self):
  """Getter method for east bond brk probability"""
  return self._eastbrkprob

def set_eastbrkprob(self, newbrk):
  """Setter method for east bond brk probability"""
  self._eastbrkprob = newbrk

def set_age(self, newage):
  """setter method for age"""
  self._age = newage

def reset_eastbrkprob(self) :
  """reset east brk probability to -1 
      to be called when single monomer is broken from polymer"""
  self.set_eastbrkprob(-1)


### Polymer Compatibility

In [29]:
%%add_to Monomer

def get_chirality(self):
  """ Return an expression of handedness compatible with the Polymer notation
  of chirality (n_lefts, n_rights).
  """
  if self.get_handedness():
    return (1,0)
  else:
    return (0,1)

def get_length(self):
  """ Return 1. """
  return 1

def get_ee(self):
  """ Return the enantiomeric excess of the monomer. This value is always 1. """
  return 1

def get_signed_ee(self):
  """ Return the signed enantiomeric excess of the monomer.
  Returns 1 if left handed, -1 if right handed.
  """
  if self.get_handedness():
    return 1
  else:
    return -1

def generate_log(self):
  """ Return a log compatible with the reactables history. """
  return (self.get_handedness(),self.get_age())

def get_sequence(self):
  return (self.get_handedness(),)

def birthday(self):
  """ages the monomer up"""
  self._age += 1

## The Polymer class

This is our Polymer class, aka the relationships in this dating show. They contain a list of all the monomers that are inside of them. It is kept in a specific order. They are pretty open books. They can tell us how many monomers they contain, they add new members(either monomers or whole other polymers), they can remove entire groups of members after a certain index, they can check which of their bonds are homochiral and use this to calculate probability of a certain bond breaking(the weak areas in their relationship). They can use this to then tell the producers if and where they are planning on breaking off members of they ranks. Like all good relationships, they use purely mathematics and chance to determine these heart breaking decisions. They can also tell us how many left handed monomers and how many right handed monomers they contain. 

### Constructor

In [30]:
class Polymer:
  def __init__(self, monomers = []):
    """ Constructor method for Polymer.
    In addition to setting the monomers list, also calculates all break probabilities
    of the bonds it contains using reset_break_probability.
    """
    self._monomers = monomers
    #self.reset_break_probability()
    #keeps track of the age of itself(number of iterations it has survived)
    self._age = 0

  def __str__(self):
    """ str function for Polymer class. """
    return f"\nA polymer of length {self.get_length()}: {self.get_sequence()}"
  
  def __repr__(self):
    """ repr function for Polymer class. """
    return f"{self.get_sequence()}-polymer"

### Getters, Setters, and Logs

In [31]:
%%add_to Polymer


def get_monomers(self):
  """ Return list of monomers in the polymer. """
  return self._monomers


def get_length(self):
  """ Return the number of monomers in the polymer. """
  return len(self._monomers)


def get_sequence(self):
  """ Return the sequence of monomers in the polymer in a human-readable (and loggable) format. """
  sequence = []
  for monomer in self.get_monomers():
    sequence.append(monomer.get_handedness())
  return sequence

def get_age(self):
  """
  Getter method for the age of the polymer
  """
  return self._age


def set_age(self, newage):
  """
  Setter method for the age of the polymer
  """
  self._age = newage

def set_monomers(self, new_monomers):
  """ Set the monomers in the polymer to new_monomers. """
  self._monomers = new_monomers

def get_avg_age(self):
  """
  calculates the average age of the monomers in the polymer
  """
  totalage = 0
  for n in range(self.get_length()):
    totalage += self.get_monomers()[n].get_age()
  avgage= totalage/self.get_length()
  return avgage


def get_chirality(self):
  """ Return the chiralities of the monomers within the polymer.

  RETURN: tuple
  (number_lefts, number_rights)
  """
  #Get the monomer handednesses
  sequence = self.get_sequence()
  #Count the number of trues
  n_true = sum(sequence)
  #The number of falses is the total minus the number of trues
  #Return that.
  return (n_true, len(sequence) - n_true)

def birthday(self):
  """
  Ages up the polymer and all of the monomers in the polymer
  """
  self._age += 1
  for n in range(self.get_length()):
    self.get_monomers()[n].birthday()
    

def get_signed_ee(self):
  lefts, rights = self.get_chirality()
  return (lefts - rights) / self.get_length()

def get_ee(self):
  lefts, rights = self.get_chirality()
  return abs((lefts - rights) / self.get_length())

  
def generate_log(self):
  """ Return a log tuple formatted simply as the list of boolean handednesses of the monomers contained
  in the polymer. Will be heavier than holding data about the polymer like in the other log format,
  but should allow access to more information and more kinds of information.
  """
  return (self.get_sequence(),self.get_age())

### Functionality methods

In [32]:
%%add_to Polymer

def append(self, other):
  """
  Adds a reactable (monomer or polymer) to the end of the Polymer.
  If the added reactable is a polymer, append() deletes it after taking its monomers.
  """
  #Check the type of thing to add
  if isinstance(other, Monomer):
    #Monomers can just be appended to the list
    self._monomers.append(other)
  elif isinstance(other, Polymer):
    #For polymers we need to extend rather than append, to avoid nested lists
    self._monomers.extend(other.get_monomers())
    #Then once the monomers are safely in their new polymer, the old one is deleted
    del other

def brkloc(self):

  #makes a list of all the indices of monomers in the polymer except 
  #for the last one
  indices = []
  for n in range(self.get_length() - 1):
    indices.append(n)
  #puts indices in random order
  random.shuffle(indices)
  #tests if each indices will break
  for index in indices:
    #gets the breakprobability for monomer at n index
    brkprob = self._monomers[index].get_eastbrkprob()
    rand = get_rand()
    # checks if the random number is less than the brkprob then the index of
    #the monomer is returned from the method
    #indicating this polymer breaks at the east bond of this monomer
    if(brkprob > rand):
      #print(self)
      #print(break_prob_lookup_table[tuple(self.get_sequence())])
      #print(index)
      #print("brkprob is:")
      #print(brkprob)
      #print("rand is")
      #print(rand)
      #print("now I go to brk polymer")
      return index
  return None

def removeright(self,location):
  """
  Removes all monomers to the right (east) of a given index
  """
  #creates a list of monomers in the polymer to the left, and including,
  #the monomer at the location(index) provided
  newList = self._monomers[0:location+1]

  #makes a list of the monoers to the rightof the monomer at the 
  #index provided
  removed = self._monomers[location+1:]

  #sets the polymers monomers to newList(the monomers to the left
  #of  and the index provided)
  self.set_monomers(newList)
  #print("after breaking ractable is:",self)
  #print("after breaking removed is:",removed)

  #returns the monomers that were removed, everything the right
  #of the index provided
  return removed

### Homochirality handling

In [33]:
%%add_to Polymer
def easthomochiralcheck(self,numbermonomer):
  """
  Takes the index of a monomer within the Polymer and returns whether its east bond is homochiral.
  """
  #First check that the index isn't out of bounds 
  if (0 > numbermonomer or numbermonomer >= self.get_length()-1):
    return False
  #get the handedness of this monomer and its east neighbor, and return whether or not they're equal (bool)
  return self._monomers[numbermonomer].get_handedness() == self._monomers[numbermonomer+1].get_handedness()

def easthomochiralbiascheck (self, numbermonomer):
  """ this method can be used in place of the east homochiral check method
  returns false if not homochiral, 3 if homochiral left, 5 if homochiral right"""

  #First check that the index isn't out of bounds or if not homochiral
  if (0 > numbermonomer or numbermonomer >= self.get_length()-1 or not self.easthomochiralcheck(numbermonomer)):
    return False
  
  #otherwise check if they are homochiral left or right
  elif self._monomers[numbermonomer].get_handedness():
      #means left homochiral
      return 3
  elif not self._monomers[numbermonomer].get_handedness():
      #means right homochiral
      return 5
  

def eastbondbreakprobability(self,numbermonomer):
  """
  Takes the index of a monomer within the Polymer
  Returns the probability that the monomer's east bond will break
  returns -3 if the monomer has no east bond
  """
  ##############################
  #Override to return an equal break chance for every bond in the polymer
  #Simply comment out this line to get the regular function
  #return (0.3)
  ##############################

  #First check if the monomer is the last one in the polymer
  if (numbermonomer >= self.get_length()-1):
    #If so, it doesn't have an east bond to break, so the probability is 0(return -3)
    return -3
    
  #now we initialize brk probability (brk)
  brk = (BASE_BOND_BREAK_PROBABILITY) #* (LENGTH_FACTOR**(self.get_length()/N))
  #check if the east bond is homochiral
  if (self.easthomochiralcheck(numbermonomer)):
    #if so multiply it by homochiral break factor (shrinks probability)
    brk *= HOMOCHIRAL_BREAK_FACTOR
    #goes through method which checks and calculates benefit of all homochiral neighbors
    brk = self.checkforhomochiralneighbors(numbermonomer,brk,HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR)
  #end def: returns break probability of monomers east bond
  return brk

def biaseastbondbreakprobability(self,numbermonomer):
  """
  ***left-right sensitivity****
  Takes the index of a monomer within the Polymer
  Returns the probability that the monomer's east bond will break
  returns -3 if the monomer has no east bond
  """
  ##############################
  #Override to return an equal break chance for every bond in the polymer
  #Simply comment out this line to get the regular function
  #return (0.3)
  ##############################
  
  #First check if the monomer is the last one in the polymer
  if (numbermonomer >= self.get_length()-1):
    #If so, it doesn't have an east bond to break, so the probability is 0 (returns -3)
    return -3
  
  #initialize the brk probability
  brk = (BASE_BOND_BREAK_PROBABILITY) * (LENGTH_FACTOR**(self.get_length()/N))

  #check if the east bond is homochiral left
  if (self.easthomochiralbiascheck(numbermonomer) == 3):
    brk *= HOMOCHIRAL_BREAK_FACTOR_LEFT

    #run through function that recalculates brk based on benfits of homochiral neighbors
    brk = self.checkforhomochiralneighbors(numbermonomer, brk, HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT)

  #otherwise checks if bond is homochiral right
  elif (self.easthomochiralbiascheck(numbermonomer) == 5):
    brk *= HOMOCHIRAL_BREAK_FACTOR_RIGHT

    #run through function that recalculates brk based on benfits of homochiral neighbors
    brk = self.checkforhomochiralneighbors(numbermonomer,brk, HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT)
   
  return brk

def checkforhomochiralneighbors(self, numbermonomer, brk, neighborfactor):

  """
  helps all the versions of the eastbond break calculating functions
  takes in a brk probability a number monomer and the desired neighbor improvement factor
  and calculates how the neighbors a bond will increase the strength of the bond
  it then returns the new brk prob to the eastbond brk function
  """
  brk = brk
  j = numbermonomer + 1
  #going right to check for homochiral neighbors
  while(self.easthomochiralcheck(j)):
    #calculates decrease to brk prob for a neighbor of that distance for the bond
    brk *= 1 - (neighborfactor**abs(j - numbermonomer))
    j += 1

  #going left to check for homochiral neighbors
  j=numbermonomer-1
  while(self.easthomochiralcheck(j)):
    #calculates decrease to brk prob for a neighbor of that distance for the bond
    brk *= 1-(neighborfactor**abs(numbermonomer - j))
    j -= 1

  return brk


def poissonbreakprobability(self,numbermonomer):
  """
  Takes the index of a monomer within the Polymer
  Returns the probability that the monomer's east bond will break
  based on Poisson distribution
  returns -3 if the monomer has no east bond
  """
  #print("POISOOOOONNNNN")
  #First check if the monomer is the last one in the polymer
  if (numbermonomer >= self.get_length()-1):
    #If so, it doesn't have an east bond to break, so the probability is 0(return -3)
    return -3

  #With that out of the way, we initialize brk to the base probability of breaking a bond
  brk = (BASE_BOND_BREAK_PROBABILITY) * (LENGTH_FACTOR**(self.get_length()/N))

  #keeps track of the number of the total consecutive homochiral bonds
  homochiralcount=0 

  #check if the east bond is homochiral
  if (self.easthomochiralcheck(numbermonomer)):
    #if so homochiral count increases by 1
    homochiralcount +=1
    #brk gets benefited by the poisson distribution of homochiral count 
    brk *= poissonequation(homochiralcount)

    #set j to monomer to right
    j = numbermonomer+1

    #going right to check for neighboring homochirality
    while(self.easthomochiralcheck(j)):
      #if homochiral increases count
      homochiralcount += 1
      #recalculates brk prob
      brk *= poissonequation(homochiralcount)

      #check next monomer
      j += 1

    #going left to check for neighboring homochirality
    j=numbermonomer-1
    homochiralcount = 1
    while(self.easthomochiralcheck(j)):
      homochiralcount += 1
      brk *= poissonequation(homochiralcount)
      j -= 1

  return brk


def reset_break_probability(self, method):
    
  """ 
  resets the break probabilities of the monomers in a 
  sequence. 1)checks lookup table to see if similar polymer values
  have been caluclated, otherwise calculates values and adds
  polymer to lookup table 2)sets values to monomers.
 
  """
  #store self as list of booleans to be added to lookup table
  polyseq = self.get_monomers().copy()
  for n in range(self.get_length()):
    polyseq[n] = polyseq[n].get_handedness()

  ##########################
  # LEFT/RIGHT SENSITIVITY #
  ##########################
  # All stored sequences start with True. If the one being requested doesn't, invert so that it does
  if not method == "bias":
    if polyseq[0] == True:
      polyseq = [not mono for mono in polyseq]

  #make polyseq a tuple able to be added as an index of a dictionary  
  polyseq = tuple(polyseq)

  # If the sequence is already in the table (dict), retrieves its previously calculated break probabilities
  if polyseq in break_prob_lookup_table:
    the_probs = break_prob_lookup_table[polyseq]
    
  else: # Otherwise calculate and add to the lookup table
    # Calculate the break probabilities
    the_probs = self.calculatebrkprob(method)
    #and add it to the lookup table
    break_prob_lookup_table[polyseq] = the_probs
  #set monomers eastbond brk probs to correlated values stored in the_probs
  for n in range (self.get_length()):
    self._monomers[n].set_eastbrkprob(the_probs[n])

  #end def: polymer break probabilities recalculated and added
  #to lookup table (if not already there)

def calculatebrkprob(self, method):
    
  """
  generates and returns a list of break probabilities for the monomers at each index of
  a polymer. This method can be altered to calculate normally
  with left/right sensitivity or with a poisson distribution
  
  Calculation methods: [bias, standard, poisson]

  """

  #initialize brk_probs as an empty list
  brk_probs = []
  
  #checks if polymer is of length one -meaning it is lone monomer
  if self.get_length() == 1 :
    #if so adds -1 to list(brk prob value assigned to individual monomers)
    brk_probs.append(-1)
    
  else:
    #otherwise goes through all the indices of the polymer,calculates its brk probability, and appends it to brk_probs 
    for n in range(self.get_length()):

      #############################################################
      #LEFT RIGHT SENSITIVITY/POISSON EQUATIONS CHOICE.           #
      #############################################################

      if method == "bias":
        brk_probs.append(self.biaseastbondbreakprobability(n))

      elif method == "standard":
        brk_probs.append(self.eastbondbreakprobability(n))

      elif method == "poisson":
        brk_probs.append(self.poissonbreakprobability(n))

      else:
        raise ValueError(f"'{method}' is not a recognized calculation method. Please use one of ['bias', 'standard', 'poisson'].")


  #returns in order list of brk probs for the east bond of all the monomers in a polymer
  return brk_probs


## The Reactables class

This is the Reactables class aka the stylish little bag we keep our contestants in. As all good dating shows, we define our members by the relationship they are in. So a single monomer will be addd to the bag alone, but a monomer in a polymer will not be. Rather each polymer will be recorded once. Its like if you have charlie, ben, and jack. Jack and Ben are in a relationship so they are jointly called Jen. If these three were in the reactable bag they would be recognized as two things, charlie and Jen(Ben+JAck). The reactable class is the heart and center of our game- I mean dating- show. Its here we ask all the members just the right questions to get them falling in love and breaking hearts, and then we are curteous enough to do all the splitting and merging for them.

### Constructor

In [34]:
class Reactables:

  def __init__(self, reactables = []):
    """
    Constructor class for Reactables object
    Accepts list of reactables or defaults to empty list
    Reactables functions as a bin for reactable objects monomer and polymer
    Methods consist mostly of list handling
    """
    self._reactables = reactables
    #The reactables bag also handles remembering its history
    self._history = []
    #list of the age of the polymers in the reactables bag. 
    self._agehistory = []
    self._lookup = {}
    self._hist_stats = pd.DataFrame()



  def __str__(self):
    """
    str method for Reactables class
    """
    return str(self.get_reactables())

### Getters and Setters

In [35]:
%%add_to Reactables

#################
#GETTERS/SETTERS#
#################

def get_reactables(self):
  """
  Getter method for reactables in Reactables
  Returns a list of the objects currently in the Reactables bag
  """
  return self._reactables

def get_history(self):
  """ Return the history of the reactables bag.
  """
  return self._history

def set_history(self, new_hist):
  """ Set the history to now_hist. """
  self._history = new_hist

#def get_leftrighthistory(self):
  #return self._leftrighthistory

def get_agehistory(self):
  """
  Getter method for the age history of the reactables bag
  """
  return self._agehistory
  
def get_count(self):
  """
  Getter method for Reactables class
  Returns the number of objects in the Reactables bag
  """
  return len(self.get_reactables()) 


def set_reactables(self, new_list):
  """
  Setter method for the Reactables class
  Sets the reactables bag to a new list
  Used mostly as a helper function in bonding
  """
  self._reactables = new_list

def set_agehistory(self, newagehistory):
  self._agehistory = newagehistory


def get_bond_chance(self, reactable1, reactable2):
  """
  Returns the probability that two given reactables in the reactables bag will bond
  """
  return BOND_PROB

def get_free_monomers(self):
  return [r for r in self.get_reactables() if isinstance(r, Monomer)]

def get_polymers(self):
  return [r for r in self.get_reactables() if isinstance(r, Polymer)]

def get_bound_monomers(self):
  bound = []
  for p in self.get_polymers():
    bound.extend(p.get_monomers())
  return bound


def get_stats(self):
  return self._hist_stats

def set_hist_stats(self,df):
  self._hist_stats = df

def get_lookup(self):
  return self._lookup

def add_stat_lookup(self, key, value):
  self._lookup[key] = value

def add_log(self, log):
  (self._history).append(log.copy())


### Functionality methods

#### Top level functionality

In [36]:
%%add_to Reactables

def refill(self, pool_size):
  #Do we have less than the required number of things in the pool?
  dearth = pool_size - self.get_count()
  #If so,
  if dearth > 0:
    new = []
    #Make as many monomers as we need to get back up to the riquired amount
    for n in range(dearth):
      new.append(Monomer())
    #And add them to the reactables bag
    self.add(new)

def randomize_reactables(self):
  """
  Randomizes the order of the reactables list
  """
  #Get the current reactables
  reactables = self.get_reactables()
  #Shuffle them
  random.shuffle(reactables)
  #And reset the reactables list to the new sorted list
  self.set_reactables(reactables)

def do_the_thing(self, method):
  """
  Handle a single iteration of the reactables.

  """
  #We keep track of bonding by storing the most recent reactable to choose to bond as the 'bachelor.'
  bachelor = None
  #We need a copy of the reactables because indices will change as soon as we start doing stuff
  reactables = copy.copy(self.get_reactables())
  #Iterate through each reactable
  for item in reactables:
    #Roll a random number in (0,1) to compare against our probabilities
    roll = get_rand()
    #If our reactable is a monomer...
    if isinstance(item, Monomer):
      #We get our breaking and bonding chances (parameters now, may become functions later?)
      brk = POOF_CHANCE
      bond = BOND_PROB
      #If we roll to break the monomer
      if 0 < roll <= brk:
        #Just delete it
        del item
      #If we roll to bond the monomer
      elif brk < roll <= brk+bond:
        #Check if there's a reactable waiting to bond
        if bachelor == None:
          #If not, make this monomer the bachelor. Someone will come along to bond later.
          bachelor = item
        else:
          #If there is someone waiting, bond them together and reset the bachelor chair to empty.
          #print("I have gone to bonnnnnnddddddd!!!!")
          #print(bachelor)
          #print(item)
          self.bond_pair(bachelor, item, method)
          bachelor= None
      else:
        continue
    #If the reactable is a polymer instead...
    elif isinstance(item, Polymer):
      #Choose whether the polymer will break or bond. It's 50-50 right now.
      if roll >= 0.33:
        #This is the same bonding logic as for the monomer. It's not in a helper function because it needs to be able to access bachelor.
        if bachelor == None:
          bachelor = item
        elif (type(bachelor) != Polymer or type(item) != Polymer) and type(item) == type(bachelor) and (FUSION == False):
          #print("I have gone to bonnnnnnddddddd!!!!")
          #print(bachelor)
          #print(item)
          self.bond_pair(bachelor, item, method)
          bachelor = None
        elif FUSION:
          self.bond_pair(bachelor,item, method)
          bachelor = None
      #Here's what happens if the polymer is chosen to check breaking
      else:
        #It finds its break location
        #print("I am going to fing my brk location!!!")
        break_spot = item.brkloc()
        #Which might turn out to be nowhere.
        if break_spot != None:
          #If somewhere does break, call break_polymer to handle the breaking
          #print(break_spot)
          self.break_polymer(item,break_spot,method)
    else:
      #You never know what might end up in your reactables bag
      raise ValueError("This thing is neither a monomer or a polymer. What?!")

def iterate(self,size,iteration_number,method):
  """
  Handles full sequence of iteration
  """
  self.randomize_reactables()
  self.do_the_thing(method)
  self.log(iteration_number)
  self.ageup()
  #self.record_leftrighthomochiral()
  self.refill(size)

def simulate(self, poolsize, iterations, method="standard"):
  for n in range(iterations):
    self.iterate(poolsize, n, method)
  self.parse_history()


#### Functionality helpers

In [37]:
%%add_to Reactables


def add(self, new_reactables):
  """
  Adds a list of reactables to the reactables bag
  Built on extend(); argument must be iterable
  """
  #print("list of new Reactbles", new_reactables)
  #print("list of get Reactables:",self.get_reactables())
  self.get_reactables().extend(new_reactables)
  #print("reactables list is now",self.get_reactables())


def subtract(self, removables):
  """
  Removes a list of reactables from the reactables bag
  Built on list comprehension; argument must be iterable
  """
  self.set_reactables([reactable for reactable in self.get_reactables() if not (reactable in removables)])


def bond_pair(self, west, east, method):
  """
  Accepts two reactables from the bag
  Bonds them together, handling deletion of any emptied polymer
  """

  #this is my attempt to get the ages working properly the idea is:
  if type(west) == Polymer and type(east) == Polymer:
    print("TWWWWOOOOO POLYMERSSSSSS ARE BONNNNDINGNGNGNG")
  #if both are polymers or both are monomers:
  if (type(west) == type(east) and type(west) == Polymer):
    #set the new age to the maximum age of the polymers
    age = max(west.get_age(), east.get_age())
  
  if (type(west) == type(east) and type(west) == Monomer):
    #set age to 0 it is a brand new polymer
    age = 0

  #if only one is a polymer:
  elif isinstance(west, Polymer) and isinstance(east, Monomer):
    #set the age to the age of the polymer
    age = west.get_age()
  elif isinstance(west, Monomer) and isinstance(east, Polymer):
    # set the age to the age of the polymer
    age = east.get_age()

  #Take your two reactables and check if the west one is a monomer or a polymer 
  if isinstance(west, Polymer):
    #If it's a polymer, all you've got to do is append the east item and then take it out of the bag
    west.append(east)
    self.subtract([east])
    #recalculate the breakprobabilities of the bonds in polymer
    west.reset_break_probability(method)
    #set new polymer to age that was calculated before
    west.set_age(age)
    #print("I have just bonded are we good with my break probs?/?????")
    #print(west)
    #brkprobs = []
    #for n in range(west.get_length()):
      #brkprobs.append(west.get_monomers()[n].get_eastbrkprob())
    #print(brkprobs)
    #print("compared toooo")
    #print(break_prob_lookup_table[tuple(west.get_sequence())])
    #print("#####################")

  if isinstance(west, Monomer):
    #If west friend is a Monomer, then make a new polymer containing west
    newpoly = Polymer([west])
    #Add the east things to it
    newpoly.append(east)
    #Remove both the west monomer and the east thing
    self.subtract([east])
    self.subtract([west])
    #And put the new polymer into the reactables bag
    self.add([newpoly])
    #recalculates break probability
    newpoly.reset_break_probability(method)
    #set new polymer to age that was calculated before
    newpoly.set_age(age)
    #print("I have just bonded are we good with my break probs?/?????")
    #print(newpoly)
    #brkprobs = []
    #for n in range(newpoly.get_length()):
      #brkprobs.append(newpoly.get_monomers()[n].get_eastbrkprob())
    #print(brkprobs)
    #print("compared toooo")
    #print(break_prob_lookup_table[tuple(newpoly.get_sequence())])
    #print("#####################")



def break_polymer(self, polymer, brk_location, method):
  #breaks polymer at given location and creates a new polymer of the 
  #monomers removed
  #when polymer is made-breakprobabilities are calculated
  age = polymer.get_age()
  #print(polymer)
  newPolymer = Polymer(polymer.removeright(brk_location))
  #if(newPolymer._monomers[0].get_handedness() and polymer._monomers[-1].get_handedness()):
    #print ("left breaking when literally impossible????")
  #if(not newPolymer._monomers[0].get_handedness() and not polymer._monomers[-1].get_handedness()):
   # print("right breaking")
  #print("I have broken myself at ")
  #print(brk_location)
  #print(polymer)
  #print(newPolymer)
  polys = [newPolymer, polymer]
  #resets break probabilities in polymers
  #goes throught the two new polymers
  for poly in polys:
    poly.set_age(age)
    #resets break probabilities in polymers
    poly.reset_break_probability(method)
    #brkprobs = []
    #for n in range(poly.get_length()):
      #brkprobs.append(poly.get_monomers()[n].get_eastbrkprob())
    #print(brkprobs)
    #print("compared tooooooooo")
    #print(break_prob_lookup_table[tuple(poly.get_sequence())])
    #print("############################")
    #if poly.get_length()==0:
      #print("whyyyyyyyyyyy is this happening????")
    #checks if they are of length 1
    if poly.get_length() <= 1:
      #adds them to reactable as a single monomer
      self.add(poly.get_monomers())
      #subtracts polymer from reactable list
      self.subtract([poly])
      del poly
    #checks if the polymer is not in the reactables list
    elif (poly not in self.get_reactables()):
      #add polymer to reactable list
      self.add([poly])

    



### Current state data methods

In [38]:
%%add_to Reactables

def get_overall_chirality(self):
  """ Return the total quantity of left- and right-handed monomers in the reactable
    pool, including monomers within polymers.

  RETURN tuple
  (Left-handed_quantity, Right-handed_quantity)
  """
  left_count = 0
  right_count = 0
  for reactable in self.get_reactables():
    if isinstance(reactable, Monomer):
      if reactable.get_handedness():
        left_count += 1
      else:
        right_count += 1
    elif isinstance(reactable, Polymer):
      l, r = reactable.get_chirality()
      left_count += l
      right_count += r
    else:
      print(f"{reactable}, which is a {type(reactable)}")
      raise ValueError
  return (left_count, right_count)

def get_polymer_chirality(self):
  """ Return the total quantity of left- and right-handed monomers in the polymers
  within the reactable pool.

  RETURN tuple
  (Left-handed_quantity, Right-handed_quantity)
  """
  left_count = 0
  right_count = 0
  for reactable in self.get_reactables():
    if isinstance(reactable, Polymer):
      l, r = reactable.get_chirality()
      left_count += l
      right_count += r
  return (left_count, right_count)

def get_free_proportion(self):
  """Return the proportion of free monomers to total population"""
  return len(self.get_free_monomers())/self.unit_count()

def unit_count(self):
  """return the total number of monomers in the bag, both free and bound"""
  bound = 0
  for p in self.get_polymers():
    bound += p.get_length()
  return len(self.get_free_monomers()) + bound

def get_polymer_ee(self):
  ee = []
  for polymer in self.get_polymers():
    ee.append(polymer.get_signed_ee())
  return ee

def avg_ee_by_length(self):
  """ Return a list containing the enantiomeric excess of all reactables sorted by size.
  That is:
  [(ee of monomers),(ee of 2-mers),(ee of 3-mers) etc]
  """
  output = []
  for n in range(1,self.max_length()+1):
    group = self.get_reactables_by_length(n)
    ees = []
    # print(f"For length {n} I got {len(group)} reactables.")
    if len(group) > 0:
      # eestr = ""
      for r in group:
        ees.append(r.get_ee())
        # eestr += f", {r.get_ee()}"
      output.append(np.mean(ees))
      # print(f"The ee's of this length were {eestr}.\nThe average ee was {np.mean(ees)}.")
    else:
      output.append(0)
  return output

def get_reactables_by_length(self,length):
  if length <= 1:
    #return the monomers
    return self.get_free_monomers()
  else:
    return [p for p in self.get_polymers() if p.get_length() == length]

def max_length(self):
  """ Return the length of the longest polymer in the reactables bag.
  """
  length = 0
  for p in self.get_polymers():
    if p.get_length() > length:
      length = p.get_length()
  return length

def ageup(self):
  """
  Method that ages up every reactable in the reactables bag
  """
  for reactable in self.get_reactables():
    reactable.birthday()
    

def logagehistory (self):
  """
  Method that logs the age of the molecules in the current iteration. Only logs the polymers ages.
  """
  log = []
  #gets all the polymers in an iteration
  polymers = self.get_polymers()
  #Goes through polymers
  for molecule in polymers:
    #appends age to log
    log.append(molecule.get_age())
    #returns log of a list of ages of the polymers of current iteration
  return log
  



### History handling methods

In [39]:
%%add_to Reactables

def log(self, iteration):
  """ Translate current reactables into a loggable list of tuples.
  This log keeps the full sequences of the polymers without digesting the data, 
  so it should be more flexible as we more forward with pulling new information
  out of this simulation. 
  There is one major piece of information lost, however, which is the identities
  of the individual polymers and monomers. """

  # Column labels for History Handling #
  Type = "Type"                   # Monomer or polymer
  Length = "Length"               # How many monomers in the thing
  nLefts = "#Lefts"               # How many left monomers in the thing
  nRights = "#Rights"             # How many right monomers in the thing
  Sequence = "Sequence"           # The string sequence of the thing
  nLhomo = "#LeftHomochiral"      # The number of left homochiral bonds in the polymer
  nRhomo = "#RightHomochiral"     # The number of right homochiral bonds in the polymer
  nHomo = "#Homochiral"           # The overall number of homochiral bonds in the polymer
  sEE = "Signed ee"               # The signed enantiomeric excess of the polymer (+ if more True)
  pcHomo = "%Homochirality"       # The proportion of bonds in the polymer that are homochiral
  pcLhomo = "%LeftHomochirality"  # The proportion of bonds in the polymer that are left homochiral
  pcRhomo = "%RightHomochirality" # The proportion of bonds in the polymer that are right homochiral
  Iter = "Iteration"              # The iteration number at which the item is found
  Age = "Age"                     # The age of the item


  for r in self.get_reactables():
    item = r.get_sequence()
    age = r.get_age()
    # Translate
    sequence = self.standard_form(item)

    #Is it in the lookup table?
    if sequence not in self.get_lookup():

      # If not, check what kind of data to gether
      if len(item) == 1: # It's a monomer
        # Get the info
        lr = self.count_LR(item)
        # And put it in the lookup table
        self.add_stat_lookup(sequence, {Type: 'Monomer', 
                                        Length:1, 
                                        nLefts:lr[0], 
                                        nRights:lr[1],
                                        Sequence:sequence})
      elif len(item) > 1: # It's a polymer
        # Get basic info
        length = len(item)
        lr = self.count_LR(item)
        bonds = self.homochiral_bond_counts(item)
        total_homos = bonds[0]+bonds[1]
        signed_ee = (lr[0]-lr[1])/(lr[0]+lr[1])
        homochirality = total_homos/(length-1)
        lhomochirality = bonds[0]/(length-1)
        rhomochirality = bonds[1]/(length-1)
        # And put it in the lookup table
        self.add_stat_lookup(sequence, {Type: 'Polymer', 
                                        Length: length, 
                                        nLefts:lr[0], 
                                        nRights:lr[1],
                                        nLhomo:bonds[0],
                                        nRhomo:bonds[1],
                                        nHomo:total_homos,
                                        sEE:signed_ee,
                                        pcHomo:homochirality,
                                        pcLhomo:lhomochirality,
                                        pcRhomo:rhomochirality,
                                        Sequence:sequence})
      else:
        raise ValueError("There's something with length 0 in your history.")

    # Now that the data is searchable...
    # Log it
    new_log = self.get_lookup().get(sequence)
    new_log[Iter] = iteration
    new_log[Age] = age
    self.add_log(new_log)


def parse_history(self):
  """ Create an array of plottable information from the history log. """
  parsed_hist = pd.DataFrame(self.get_history())
  self.set_hist_stats(parsed_hist)


#### Low level info functions

In [40]:
%%add_to Reactables

## New ones ##

  
# Functions for getting the info we want to log
    
# Number of right and left monomers in a polymer (works on monomers too)


def count_LR(self, log):
    """ Return (n_Lefts, n_Rights) """
    if L and not R:
        return (sum(log), len(log)-sum(log))
    elif R and not L:
        return (len(log)-sum(log), sum(log))
    else:
        raise ValueError("Your L's and R's are screwed up somehow.")
    


# HOMOCHIRALITY SEQUENCES #

def homochirality_sequence(self, log):
  """ Return a boolean list of bonds within a given logged polymer, True if the bond is
      homochiral. """
  return [i[0]==i[1] for i in list(more_itertools.pairwise(log))]

def _is_L_True(self, log):
    return log[:-1]


def T_homochirality_sequence(self,h_seq,L_seq):
    """ Return a boolean list of bonds within a polymer, true if the bond is 
    a homochiral bond between two True monomers. Parameters are the outputs of 
    homochirality_sequence() and _is_L_True(). """
    return eAnd(h_seq,L_seq)

# HOMOCHIRAL BOND COUNTS #
    

def homochiral_bond_counts(self,log):
    """ Return (number of left homochiral bonds, number of right homochiral bonds) """
    homo = self.homochirality_sequence(log)
    west_true = log[:-1]
    true_homochiral = eAnd(homo,west_true)
    west_false = [not m for m in west_true]
    false_homochiral = eAnd(homo,west_false)
    if L and not R:
        return (sum(true_homochiral), sum(false_homochiral))
    elif R and not L: 
        return (sum(false_homochiral), sum(true_homochiral))
    else:
        raise ValueError("Your L's and R's are screwed up somehow.")


def get_polymer_chain_lengths(self,polylog):
  """ Return the lengths of the homochiral chains in a given polymer log. """
  count = 1
  lengths = [] 
  for n in range(1,len(polylog)):
    if polylog[n] == polylog[n-1]:
      count += 1
    else:
      lengths.append(count)
      count = 1
  lengths.append(count)
  return lengths
    

def standard_form(self,poly):
  if len(poly) == 0:
    return ''
  return ''.join(['L' if m else 'R' for m in poly])
    



## Old ones ##

def hist_get_polymer_homochirality_of_bonds(self,polylog):
  """ Return a boolean list of bonds within a given logged polymer, True if the bond is
      homochiral. """
  return [i[0]==i[1] for i in list(more_itertools.pairwise(polylog))]

def hist_get_polymer_ee(self,polylog):
  """ Return the (signed) enantiomeric excess of a logged polymer. """
  length = len(polylog)
  n_True = sum(polylog)
  n_False = length - n_True
  return (n_True - n_False) / length

def hist_get_polymers(self,iteration):
  """ Return a list of the polymer logs in an iteration. This is just the iteration
  but without the monomers. """
  return [r for r in iteration if len(r) > 1]

def hist_count_longest_homochiral_chain(self,polylog):
  """ Return the length of the longest homochiral chain given the log of a polymer. """
  previous = None
  count = 1
  longest = 1
  for monomer in polylog:
    if monomer == previous:
      count += 1
    else:
      longest = max(count, longest)
      count = 1
    previous = monomer
  longest = max(count, longest)
  return longest

def hist_get_polymer_chain_lengths(self,polylog):
  """ Return the lengths of the homochiral chains in a given polymer log. """
  count = 1
  lengths = [] 
  for n in range(1,len(polylog)):
    if polylog[n] == polylog[n-1]:
      count += 1
    else:
      lengths.append(count)
      count = 1
  lengths.append(count)
  return lengths

def hist_get_iteration_chain_lengths(self,iteration):
  """ Return the lengths of all the homochiral chains in polymers in a given iteration. """
  polymers = self.hist_get_polymers(iteration)
  chain_lengths = []
  for polymer in polymers:
    chain_lengths.extend(self.hist_get_polymer_chain_lengths(polymer))
  return chain_lengths

# Other new ones #

def hist_get_leftright_homochiral_count_iteration(self,iteration):
  """
  returns a tuple (l,r)of the total number of left homochiral bonds and right homochiral bonds in an iteration 
  """
  #initilize l to keep track of the number of homochiral bonds
  l=0
  #initilize r to keep track of the number of homochiral bonds
  r=0

  #run through reactables in the iteration
  for reactable in iteration:
     #add the left homochiral bond count in each reactable
     l += self.hist_get_leftright_homochiral_count_polymer(reactable)[0]
     #add the right homochiral bond count in each reactable
     r += self.hist_get_leftright_homochiral_count_polymer(reactable)[1]

  #place l and r in a tuple
  leftrightcount=(l,r)

  #return the tuple
  return leftrightcount

def hist_get_leftright_homochiral_count_polymer(self,polymer):
  """
  returns a tuple (l,r)of the total number of left homochiral bonds and right homochiral bonds in an polymer 
  """

  #initialize l to keep track of the number of left homochiral bonds
  l=0
  #initialize r to keep trac of the number of right homochiral bonds
  r=0

  #go through the monomers in the polymer
  for monomer in range(len(polymer)-1):

      #check if they are homochiral
        if (polymer[monomer]== polymer[monomer+1]):
          #if so check if they are homochiral left
          if (polymer[monomer]):
            #if so add one to l
            l += 1
          #otherwise check if they are homochiral right
          elif (not polymer[monomer]):
            #if so add one to r
            r += 1
  #place l and r in a tuple
  leftrightcount= (l,r)
  #return the tuple
  return leftrightcount

def hist_get_bond_status(self,polylog):
  """ Iterate through the monomers in polylog ONCE to get number of left homochiral
  and right homochiral bonds, as well as chain lengths if at all possible. """
  for m in polylog:
    pass


### Plotting methods

In [50]:
%%add_to Reactables

def visualize_parameters(self,stamp=""):
  fig,ax = plt.subplots()

  parameters = ['POOF_CHANCE','BOND_PROB','BASE_BOND_BREAK_PROBABILITY',
                'HOMOCHIRAL_BREAK_FACTOR','HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR',
                'LENGTH_FACTOR','HOMOCHIRAL_BREAK_FACTOR_LEFT',
                'HOMOCHIRAL_BREAK_FACTOR_RIGHT',
                'HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT',
                'HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT']
  values = [POOF_CHANCE,BOND_PROB,BASE_BOND_BREAK_PROBABILITY,
            HOMOCHIRAL_BREAK_FACTOR,HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR,
            LENGTH_FACTOR,HOMOCHIRAL_BREAK_FACTOR_LEFT,
            HOMOCHIRAL_BREAK_FACTOR_RIGHT,
            HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT,
            HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT]

  ypos = np.arange(len(parameters))

  ax.barh(ypos, values, align='center')
  ax.set_yticks(ypos)
  ax.set_yticklabels(parameters)
  ax.invert_yaxis()  # labels read top-to-bottom
  ax.set_xlabel('Value')
  ax.set_title(f"{stamp}Parameter values")



def plot_signed_ee_spread(self,cmap='bone',stamp=""):
  #First make the polymerspread thingy
  poly_ee_hist = []
  for iteration in self.get_history():
    polys = [log for log in iteration if not isinstance(log,bool)]
    polydata = [log for log in polys if len(log) > 1] # All the polymers (not monomers) in the iteration
    poly_ees = [self.hist_get_polymer_ee(poly) for poly in polydata] # the EE of each polymer in the iteration
    poly_ee_hist.append(poly_ees) # Put the EE list in our plottable structure
  #Now we can make the plot
  fig,ax = plt.subplots()
  ys = []
  xs = []
  for n in range(len(poly_ee_hist)):
    y = poly_ee_hist[n]
    x = [n] * len(y)
    ys.extend(y)
    xs.extend(x)
  ax.hexbin(xs,ys,cmap=cmap)
  ax.set_title(f"{stamp}polymer spread")
  ax.set_xlabel("iteraions")
  ax.set_ylabel("ee")

def plot_ee_heatmap(self,cmap='bone',stamp=""):
  #Need to figure out how to get ees_by_length from history
  ees_by_length = []
  for iteration in self.get_history():
    ee_data = []
    #get the polymers
    polys = [log for log in iteration if not isinstance(log,bool)]
    #get the length of the longest polymer
    lengths = [len(log) for log in polys]
    try:
      biggest = max(lengths)
    except:
      biggest = 0
    for n in range(1,biggest+1):
      ees = [abs(self.hist_get_polymer_ee(log)) for log in polys if len(log) == n]
      if len(ees) > 0:
        ee_data.append(np.mean(ees))
      else:
        ee_data.append(0)
    ees_by_length.append(ee_data)
  #Need to make a new ee array with rectangular dimensions
  ee = numpy_fillna(ees_by_length)
  fig,ax = plt.subplots(dpi=150)
  im = ax.imshow(ee,cmap=cmap)
  ax.set_title(f"{stamp}ee of polymers by length through iterations")
  ax.set_xlabel("polymer lengths")
  ax.set_ylabel("iterations")
  fig.tight_layout()
  plt.show()

def plot_leftrighthomochirality(self):
  """
  Plot the percent of bonds that are left homochiral and right homochiral in each iteration
  """
 #make list to hold the number of left bonds
  L=[]
 #make list to hold the number of right bonds
  R=[]
 #make list to hold the number of iteration
  x=[]
 #integer to keep track of iterations
  i=1

 #work through history of reactables bag
  for iteration in self.get_history():

    #get the left right homochiral count in the iteration(returned as a tuple)
    leftrightcount=self.hist_get_leftright_homochiral_count_iteration(iteration)

    #initiate bonds as 0 to keep track of total number of bonds in the iteration
    bonds = 0

    #work through the polymers in iteration
    for polymer in self.hist_get_polymers(iteration):
      #add all the bonds in the a polymer to the total number of bonds
      bonds += len(polymer)- 1

    #to avoid a divide by zero error
    if bonds == 0:
      #just append 0 to L and R
      L.append(0)
      R.append(0)
    else: # append percent left homochiral bonds and percent which are right homochiral bonds
      L.append(leftrightcount[0]/bonds)
      R.append(leftrightcount[1]/bonds)
    #append the iteration to x
    x.append(i)
    #increment the iteration tracker
    i+=1
  
  #Plot the data
  plt.figure() 
  plt.plot(x,L,label="left homochiral")
  plt.plot(x,R,label="right homochiral")
  plt.ylabel("percentage of total bonds in the pool")
  plt.xlabel("Iterations")
  plt.title("Peercent of bonds in an iteration that are left or right homochiral ")
  plt.legend()
  plt.show()

def plot_homochiral_chain_lengths(self):
  """ Plot a chart of the mean length of homochiral chains present at each iteration.
  The plot generated will attempt to summarize the mean lengths of homochiral
  chains within polymers across iterations. It will also attempt to visualize the 
  varience with continuous lines plotting the standard deviation around that mean.
  
  The x axis of the resulting plot is iterations, and the y axis is length of 
  the homochiral chains. Both of these are discrete categories, but the plot will
  treat iterations as continuous to improve readability for large n.
  """
  # Get the history and read it 
  history = self.get_history()
  means = []
  stdevs= []
  maxes = []
  for iteration in history:
    # Get the chain lengths in that iteration
    iter_data = self.hist_get_iteration_chain_lengths(iteration)
    if iter_data == []:
      means.append(0)
      stdevs.append(0)
      maxes.append(0)
      #print("this happened in this iteration???whyyyyyyyyyyyyy")
      #print (iteration)
      #print (iter_data)
    else:
      means.append(np.mean(iter_data))
      stdevs.append(np.std(iter_data))
      maxes.append(max(iter_data))
  # Plot those data
  fig,ax = plt.subplots()
  ax.fill_between(np.arange(0,len(means)), [m + s for m, s in zip(means, stdevs)],[m - s for m, s in zip(means, stdevs)], alpha=0.2, label = "Means +- one standard deviation")
  ax.plot(means, 'k-', label = "Mean homochiral chain length")
  ax.plot(maxes, 'b.', label = "Max homochiral chain length")
  ax.set_xlabel("Iteration")
  ax.set_ylabel("Homochiral chain length")
  ax.legend()

def plot_homochirality_vs_length(self):
  """ Take all the polymers that ever existed in history and plot them on a
  scatter plot of length vs homochirality. Homochirality is the percent of their
  bonds that are homochiral. """
  #Get the history
  history = self.get_history()
  #Get the polymers
  x = []
  y = []
  for i in history:
    for p in self.hist_get_polymers(i):
      homochirality = sum(self.hist_get_polymer_homochirality_of_bonds(p))/len(p)
      x.append(len(p))
      y.append(homochirality)
  fig,ax = plt.subplots()
  ax.scatter(x,y)
  ax.set_xlabel("Polymer lengths")
  ax.set_ylabel("Homochirality")

def plot_age_vs_homochirality_leftright(self):
  """
  Make a plot showing the age of a polymer vs the percent of its bonds that are 
  left homochiral(shown in negative numbers) and right homochiral

  """
  #l is a list to hold the percent of bonds that are left homochiral in a polymer
  l=[]
  #r is a list to hold the percent of bonds that are right homochiral in a polymer
  r=[]
  #x is a list that holds the ages of the polymers
  x=[]

  #go through the age history in each iteration
  for iteration in self.get_agehistory():
    #go through each reactable in an iteration(only polymers are present in the age history)
    for reactable in iteration:
      #add age to the x
      x.append(reactable)
  #go through the general history
  for iteration in self.get_history():
    #go through the reactables in the iteration
    for reactable in iteration:
      #only look at the polymers
      if len(reactable) > 1:
        #get the number of left and number of right homochiral bonds in the polymer
        leftrightcount = self.hist_get_leftright_homochiral_count_polymer(reactable)
        #add the left and right count as a percentage of the total number of bonds in the reactable
        l.append((leftrightcount[0])/(len(reactable)-1))
        r.append(-1 * (leftrightcount[1])/(len(reactable)-1))
  #Plot the data
  plt.figure()
  plt.scatter(x,l, label = "left")
  plt.scatter(x,r, label="right")
  plt.xlabel("age")
  plt.ylabel("signed percent of bonds that are homochiral")
  plt.legend()
  plt.show()

def plot_age_vs_homochirality_ee(self):
  """ 
  Make a plot of the ages of a polymer vs the homochirality of the reactable. No left right sensitivity
  """
  #x keeps track of the ages of the polymers
  x=[]
  #y keeps track of the enantiomeric excess of the polymers
  y=[]
  #go through iteration of age history
  for iteration in self.get_agehistory():
    #go through every reactable in the iteration
    for reactable in iteration:
      #add age to the list
      x.append(reactable)
  #go through the general history
  for iteration in self.get_history():
    #go through the reactables in the iterations
    for reactable in iteration:
      #only go through the polymers
      if len(reactable) > 1:
        #append the enantiomeric excess of the polymer
        y.append(abs(self.hist_get_polymer_ee(reactable)))
  #plot the data
  plt.figure()
  plt.scatter(x,y)
  plt.xlabel("age")
  plt.ylabel("polymer_ee")
  plt.show()

def plot_age_vs_length(self):
  """
  Make a plot of the age of a reactable vs the length of the total homochiral 
  chains vs and the total nonhomochiral chains in the molecule.
  """
  plt.figure()
  z =[]
  #x keeps track of the ages of the reactables
  x= []
  y =[]
  ages = self.get_agehistory()
  #go through the iterations of the age history
  for iteration in self.get_agehistory():
    #go through the reactables in the iteration
    for reactable in iteration:
      #add the age of the reactable to the list
      x.append(reactable)

  #go through the iterations of the general history
  for iteration in self.get_history():
    #go through the reactables of the iteration
    for reactable in iteration:
      #only go through the polymers 
      if len(reactable) > 1 :
        #get a list of the lengths of the homochiral chain lengths in the polymer
        homochiralchainlengths =self.hist_get_polymer_chain_lengths(reactable)
        #go through the chain lengths 
        for n in homochiralchainlengths: 
          #Remove those that are of length one or less
          if n < 2:
            homochiralchainlengths.remove(n) 
        
        number =sum(homochiralchainlengths)/(len(reactable))
       # y.append(len(reactable) - number)
        z.append(number) 
  plt.figure()
 # plt.scatter(x,y, label= "of reactable") 
  plt.scatter(x,z)
  plt.xlabel("age")
  plt.ylabel("percent homochiral chain")
  plt.show()

def plot_homochiral_lengths_vs_poisson(self):
  """
  This graph visualizes the frequency of homochiral chain lengths in the history of the reactable bag.
  It graphs the frequency of different homochiral lengths in the bag vs the poisson distribution. 
  Do factor in liklihood that these  molecules will form, the frequencies are increased by a 
  factor of thenumber of bonds they have to account for the fact that having more bonds, is less likely. 
  """
  #create a dictionary to track the frequency of each length of a certain homochiral bond
  lengths = {}
  #x is a list of the lengths to go on the x axis
  x=[]
  #y is a frequency to go on y axis
  y=[]
  #p is to store values for a length
  p=[]

  #go through the iterations of general history
  for iteration in self.get_history():
    #go through the reactables of the iterations
    for reactable in iteration:
      #go through the polymers
      if len(reactable) >1: 
        #if so return a list  of the lenghts of the homochiral chains in the polymer
        for n in self.hist_get_polymer_chain_lengths(reactable):
          #only include the lengths that are greater than one
          if n > 1:
            #if the length is already in the dictiionary
            if n in lengths:
            #add one to its frequnency
              lengths[n] += 1
            else:#otherwise add it to the dictionay with a frequency of occurence of 1
              lengths[n] = 1
    #go through all of the recorded lengths
  for n in lengths:
    #the raise the frequency of them to a factor of the number of bonds in them, to
    #account for the decreased liklihood of occuring as the length increases that does not
    #have to do with homochirality
    number = (lengths[n]/len(self.get_history()))**((n/2)-1)
    #add the recalculated frequencied to the dictionaru
    lengths[n] = number
    #append the frequency to the y
    y.append(lengths[n])
    #append the lenght to x
    x.append(n)
    #plot the data
  plt.figure()
  #plot x and y together before sorting x
  plt.bar(x,y, label = "Homichiral lengths")
  #sort x so that it is in numerical order
  x.sort()
  #go through the lengths stored in x and calculate the poisson value for the length
  #and append this to list p 
  #multiply the values by 1000 so that they appear nicely on graph with frequencies
  for n in x:
    p.append(1000* poissonequation(n))
  #plot x vs p
  plt.plot(x,p,label = "poisson equation", color = 'orange')
  plt.xlabel("length")
  plt.ylabel("frequency")
  plt.title("Lambda =" + str(LAMBDA))
  plt.legend()
  plt.show()

def visualizebrkprobs(self):
  a = Monomer(hand = True)
  b = Polymer(monomers = [a])
  c = Monomer(hand = False)
  d = Polymer(monomers = [c])
  x = []
  y = []
  method = []
  length=[]
  for n in range(30):
    for z in range(b.get_length()-1):
      y.append(b.eastbondbreakprobability(z))
      method.append("Normal")
      x.append(z+1)
      length.append(b.get_length())
      y.append(b.poissonbreakprobability(z))
      method.append("poisson")
      length.append(b.get_length())
      x.append(z+1)
      y.append(b.biaseastbondbreakprobability(z))
      method.append("left")
      length.append(b.get_length())
      x.append(z+1)
      y.append(d.biaseastbondbreakprobability(z))
      method.append("right")
      x.append(z+1)
      length.append(b.get_length())
    monomer = Monomer(hand = True)
    b.append(monomer)
    monomer = Monomer(hand = False)
    d.append(monomer)
  data = {'NumberMonomer':  x,
      'HomochiralChainLength': length, 'Breakprob': y, 'Method': method}
  df = pd.DataFrame(data, columns = ['NumberMonomer',
      'HomochiralChainLength', 'Breakprob', 'Method'])
  fig = go.Figure()

  fig = px.scatter(df, x="NumberMonomer", y="Breakprob", animation_frame="HomochiralChainLength",
          size = "HomochiralChainLength", color="Method", hover_name="Method", range_x=[0,30], range_y=[-0.2,1.2], title = "Brk Probs of Homochiral chain")
  fig.show()


        



    


  


# Dashboard

In [53]:
bbbp = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
bbbp_label = ipywidgets.Label('BASE_BOND_BREAK_PROBABILITY:')
bbbp_box = ipywidgets.VBox([bbbp_label,bbbp])

hbf = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
hbf_label = ipywidgets.Label('HOMOCHIRAL_BREAK_FACTOR:')
hbf_box = ipywidgets.VBox([hbf_label,hbf])

hnif = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
hnif_label = ipywidgets.Label('HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR:')
hnif_box = ipywidgets.VBox([hnif_label,hnif])

lf = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
lf_label = ipywidgets.Label('LENGTH_FACTOR')
lf_box = ipywidgets.VBox([lf_label,lf])

hbfleft = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
hbfleft_label = ipywidgets.Label('HOMOCHIRAL_BREAK_FACTOR_LEFT:')
hbfleft_box = ipywidgets.VBox([hbfleft_label,hbfleft])

hbfright = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
hbfright_label = ipywidgets.Label('HOMOCHIRAL_BREAK_FACTOR_RIGHT')
hbfright_box = ipywidgets.VBox([hbfright_label,hbfright])

hnifleft = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
hnifleft_label = ipywidgets.Label('HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT')
hnifleft_box = ipywidgets.VBox([hnifleft_label,hnifleft])

hnifright = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
hnifright_label = ipywidgets.Label('HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT')
hnifright_box = ipywidgets.VBox([hnifright_label,hnifright])

lamb = ipywidgets.IntSlider(value=6, min=1, max=10, step=1)
lamb_label = ipywidgets.Label('LAMBDA')
lamb_box = ipywidgets.VBox([lamb_label,lamb])

n_val = ipywidgets.IntSlider(value=40, min=0, max=1000, step=5)
n_val_label = ipywidgets.Label('N')
n_val_box = ipywidgets.VBox([n_val_label,n_val])

pc = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
pc_label = ipywidgets.Label('POOF_CHANCE')
pc_box = ipywidgets.VBox([pc_label,pc])

bp = ipywidgets.FloatSlider(value=0.5, min=0, max=1, step=0.01)
bp_label = ipywidgets.Label('BOND_PROB')
bp_box = ipywidgets.VBox([bp_label,bp])

ps = ipywidgets.IntSlider(value=10, min=1, max=1000, step=1)
ps_label = ipywidgets.Label('POOL_SIZE')
ps_box = ipywidgets.VBox([ps_label,ps])

iterns = ipywidgets.IntSlider(value=10, min=1, max=1000, step=1)
iterns_label = ipywidgets.Label('ITERATIONS')
iterns_box = ipywidgets.VBox([iterns_label,iterns])

pf = ipywidgets.FloatSlider(value=1.3, min=1, max=2, step=0.1)
pf_label = ipywidgets.Label('POISSON_FACTOR')
pf_box = ipywidgets.VBox([pf_label,pf])

fusion = ipywidgets.Checkbox()
fusion_label = ipywidgets.Label('FUSION_FACTOR')
fusion_box = ipywidgets.VBox([fusion_label,fusion])

method_picker = ipywidgets.Dropdown(options=['bias','standard','poisson'],value="standard")
method_picker_label = ipywidgets.Label("Select calculation method:")
method_picker_box = ipywidgets.VBox([method_picker_label,method_picker])

button = ipywidgets.Button(description="GO!", button_style='primary')

glob_title=ipywidgets.Label("General parameters")
mon_title=ipywidgets.Label("Monomer parameters")
sim_title=ipywidgets.Label("Simulation paramaters")

globs = ipywidgets.VBox([glob_title,bbbp_box,lf_box,n_val_box])
mons = ipywidgets.VBox([mon_title,pc_box,bp_box])
sims = ipywidgets.VBox([sim_title,ps_box,iterns_box,fusion_box])

left_col = ipywidgets.VBox([mons,sims])

bias_widgets = [hbfleft_box,hbfright_box,hnifleft_box,hnifright_box]
standard_widgets = [hbf_box,hnif_box]
poisson_widgets = [lamb_box, pf_box]

dep_widgets = ipywidgets.VBox(standard_widgets)

center_col = ipywidgets.VBox([globs,button])

right_col = ipywidgets.VBox([method_picker_box,dep_widgets])

def choose(change):                        
  if change.new == "bias":
    dep_widgets.children = bias_widgets
  elif change.new == "standard":
    dep_widgets.children = standard_widgets
  elif change.new == "poisson":
    dep_widgets.children = poisson_widgets

def run_sim(b):
  global BASE_BOND_BREAK_PROBABILITY
  global HOMOCHIRAL_BREAK_FACTOR
  global HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR
  global LENGTH_FACTOR
  global N
  global LAMBDA
  global HOMOCHIRAL_BREAK_FACTOR_LEFT
  global HOMOCHIRAL_BREAK_FACTOR_RIGHT
  global HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT
  global HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT
  global POOF_CHANCE
  global BOND_PROB
  global POOL_SIZE
  global ITERATIONS
  global METHOD
  global break_prob_lookup_table
  global poisson_dict
  global POISSON_FACTOR
  global FUSION
  
  FUSION = fusion.value
  break_prob_lookup_table = {}
  poisson_dict={}
  POISSON_FACTOR = pf.value
  BASE_BOND_BREAK_PROBABILITY=bbbp.value
  HOMOCHIRAL_BREAK_FACTOR=hbf.value
  HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR =hnif.value
  LENGTH_FACTOR=lf.value
  N = n_val.value
  LAMBDA = lamb.value#enter some kind of if statement
  HOMOCHIRAL_BREAK_FACTOR_LEFT=hbfleft.value#enter some kind of if statement
  HOMOCHIRAL_BREAK_FACTOR_RIGHT=hbfright.value#enter some kind of if statement
  HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT=hnifleft.value#enter some kind of if statement
  HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT=hnifright.value#enter some kind of if statement
  POOF_CHANCE=pc.value
  BOND_PROB=bp.value
  POOL_SIZE=ps.value
  ITERATIONS=iterns.value
  METHOD = method_picker.value

  a = Reactables(make_pool(POOL_SIZE))
  a.simulate(POOL_SIZE, ITERATIONS, METHOD)
  #print(break_prob_lookup_table)
  #print(poisson_dict)

  stats = a.get_stats()

  with tb.output_to("Signed EE of Polymers by Iteration", select=True):
    tb.clear_tab()
    df = stats
    counts = df.groupby("Iteration")['Signed ee'].value_counts().reset_index(name='count')
    fig = px.scatter(counts, x="Iteration", y="Signed ee", size='count')
    fig.show()


  with tb.output_to("Proportion of Bond Types", select=False):
    tb.clear_tab()

    df = stats
    lefts = df.groupby("Iteration")["#LeftHomochiral"].sum().rename("LL")
    rights = df.groupby("Iteration")["#RightHomochiral"].sum().rename("RR")
    heteros = (df.groupby("Iteration")['Length'].sum() - df.groupby("Iteration")['Length'].count()).rename("LR")
    bondcounts = pd.DataFrame([lefts, rights, heteros]).transpose()
    bondcounts["Total"] = bondcounts["LL"] + bondcounts["RR"] + bondcounts["LR"]
    bondcounts = bondcounts.apply(lambda x : x / bondcounts["Total"])
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=bondcounts["LL"],
                        mode='lines',
                        name='left homochiral'))
    fig.add_trace(go.Scatter(y=bondcounts["RR"],
                        mode='lines',
                        name='right homochiral'))
    fig.add_trace(go.Scatter(y=bondcounts["LR"],
                       mode='lines', name='heterochiral'))
    fig.update_layout(title='Proportion of LL, RR, and LR bonds by iteration',
                    xaxis_title='Iteration',
                    yaxis_title='Proportion')
    fig.update_yaxes(range=[0, 1])

    fig.show()

  with tb.output_to("Homochirality vs Length through Time", select=False):
    tb.clear_tab()
    df = stats
    maxlen = df["Length"].max()

    fig = px.scatter(df, x="Length", y="%Homochirality",animation_frame="Iteration",
                    range_x=[-1,maxlen+1],range_y=[-0.01,1.2])
    fig.show()

  with tb.output_to("Homochirality vs Length", select=False):
    tb.clear_tab()
    df = stats
    df = df.groupby("Length")['%Homochirality'].value_counts().reset_index(name='count')
    fig = px.scatter(df, x="Length",y="%Homochirality",size='count')
    fig.update_traces(marker=dict(line=dict(color='DarkSlateGrey')),
                    selector=dict(mode='markers'))
    fig.show()

  with tb.output_to("Homochirality vs Age", select=False):
    tb.clear_tab()
    df = stats
    df = df.groupby("Age")['%Homochirality'].value_counts().reset_index(name='count')
    fig = px.scatter(df, x="Age",y="%Homochirality",size='count')
    fig.update_traces(marker=dict(line=dict(color='DarkSlateGrey')),
                    selector=dict(mode='markers'))
    fig.show()

  with tb.output_to("Visualize Break Probs", select=False):
    tb.clear_tab()
    a.visualizebrkprobs()
    x = []
    y = []
    for n in range(100):
      x.append(n)
      y.append(BASE_BOND_BREAK_PROBABILITY * LENGTH_FACTOR**(n/N))
    data = {'Length':  x,
      'BrkProb': y}
    df = pd.DataFrame(data, columns = ['Length',
      'BrkProb'])
    fig = px.line(df, x = 'Length', y = 'BrkProb', title = "Effects of Length on Break Probs")
    fig.show()

  with tb.output_to("Length by homochirality through age", select = False):
    tb.clear_tab()
    df = stats
    fig = px.scatter(df, x="Length", y="%Homochirality", animation_frame="Age", hover_name="%Homochirality", color = "Type",range_x=[-1,maxlen+1])
    fig.show()

  with tb.output_to("Age by homochirality through length", select = False):
    tb.clear_tab()
    df = stats
    fig = px.scatter(df, x="Age", y="%Homochirality", animation_frame="Length", hover_name="%Homochirality", color = "Type")
    fig.show()

  with tb.output_to("Length by Homochirality colored by Age", select = False):
    tb.clear_tab()
    df = stats
    fig = px.scatter(df, x = "Length", y ="%Homochirality",color = "Age", range_x =[-1,maxlen+1])
    fig.show()
    
  
  #with tb.output_to("Homochirality by average age", select = False):
    #tb.clear_tab()
    #df = stats 
    #f.groupby(level="Type").mean()
    #df.sortby(["%Homochirality"])
    #print(df)
    #fig = px.bar(df, x ="%Homochirality", y = "Age")
    #fig.show()

button.on_click(run_sim)

container = ipywidgets.HBox([left_col,center_col,right_col])

method_picker.observe(choose, names="value")

display(container)

plots = ["Signed EE of Polymers by Iteration","Proportion of Bond Types",
         "Homochirality vs Length through Time","Homochirality vs Length",
         "Homochirality vs Age","Visualize Break Probs","Length by homochirality through age",
         "Age by homochirality through length","Length by Homochirality colored by Age","Homochirality by average age"]
tb = widgets.TabBar(plots)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>